In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import rc
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.isri import ISRIStemmer

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

D:\new\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
#https://www.kaggle.com/datasets/asmaaabdelwahab/arabic-news-dataset
data = pd.read_csv("processed_data.csv")
data.head()

,Title,Content,Link,Category
0,تيرتل تري لابز للتقنيات الحيويه تحصل علي تمويل...,جاءت جوله التمويل التمهيديه الاولي للشركه بقيا...,https://ryadiybusiness.com/%d8%aa%d9%8a%d8%b1%...,ريادة أعمال
1,تعد دراسه الجدوي لمشروعك الناشء 6 عوامل اساسيه,تعد دراسه الجدوي متطلبا اساسيا لنجاح اي مشروع ...,https://ryadiybusiness.com/%d9%83%d9%8a%d9%81-...,ريادة أعمال
2,منح جاءزه نوبل الطب لاكتشاف مجال التهاب الكبد ...,وقالت لجنه نوبل للمره الاولي التاريخ يمكن الان...,https://arabic.sputniknews.com/science/2020100...,علوم وتكنولوجيا
3,هيءه الغذاء والدواء الامريكيه لقاح مودرنا حقق ...,نشرت الوكاله وثاءق علي الانترنت اعدها موظفوها ...,https://arabic.sputniknews.com/world/202012151...,علوم وتكنولوجيا
4,حظر بروتين الجهاز المناعي يساعد علي محاربه عدو...,دراسه حديثه قام فريق العلماء جونز هوبكنز ميديس...,https://arabic.rt.com//technology/1161864-%D8%...,علوم وتكنولوجيا


In [4]:
data.drop('Title',inplace=True,axis=1)
data.drop('Link',inplace=True,axis=1)
data.drop('Category',inplace=True,axis=1)

In [5]:
data.isnull().sum()

Content    13
dtype: int64

In [6]:
data.dropna(inplace=True)
data.shape

(8386, 1)

In [7]:
dupl = data[data.duplicated()]
if len(dupl)>0:
    data=data.drop_duplicates()
    print(len(dupl))
data.reset_index(drop=True, inplace=True)

463


In [8]:
import re

def normalize_arabic(text):
    if not isinstance(text, (str, bytes)):
        # If input is not a string or bytes-like object, convert it to string
        text = str(text)
    
    if text.startswith("ال"):
        text = text[2:]
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("[ًٌٍَُِّْ]", "", text)
    return text

In [9]:
def preprocess_arabic_Remove_stop(words):
    stop_words = set(stopwords.words('arabic'))
    preprocessed_text=[]
    words = [word for word in words if word not in stop_words]
    preprocessed_text = words

    return preprocessed_text

In [10]:
import string
def remove_punctuation(words):


    cleaned_words = [word for word in words if word not in string.punctuation]
    return cleaned_words

In [11]:
def lemma(words):

    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return lemmatized_words

In [12]:
def preprocess_text(text):
    words = word_tokenize(text)
    out = remove_punctuation(words)
    out = preprocess_arabic_Remove_stop(out)
    out = lemma(out)
    return out

In [13]:
def create_inverted_index(documents):
    inverted_index = dict()
    for doc_id, doc in enumerate(documents):
        doc = normalize_arabic(doc)
        terms = preprocess_text(doc)

        for term in terms:
            if term not in inverted_index:
                inverted_index[term] = []
            if doc_id not in inverted_index[term]:
                inverted_index[term].append(doc_id)
    return inverted_index

In [14]:
inverted_index = create_inverted_index(data['Content'])

In [15]:
def search(query, inverted_index, documents):
    query = normalize_arabic(query)
    query = preprocess_text(query)
    relevant_doc_ids = list(range(len(documents)))

    ok = 0
    print(query)

    for term in query:
        relevant_docs = []
        if term in inverted_index:
            p1 = 0
            p2 = 0
            while p1 < len(relevant_doc_ids) and p2 < len(inverted_index[term]):
                if relevant_doc_ids[p1] < inverted_index[term][p2]:
                    p1 += 1
                elif relevant_doc_ids[p1] > inverted_index[term][p2]:
                    p2 += 1
                else:
                    relevant_docs.append(relevant_doc_ids[p1])
                    p1 += 1
                    p2 += 1
            relevant_doc_ids = relevant_docs
    return relevant_doc_ids

In [16]:
search('جونز هوبكنز',inverted_index,data['Content'])
res=data.iloc[search('جونز هوبكنز',inverted_index,data['Content'])]
res.iloc[:, 0]

['جونز', 'هوبكنز']
['جونز', 'هوبكنز']


4       دراسه حديثه قام فريق العلماء جونز هوبكنز ميديس...
69      الان بدا توزيع لقاحات عديده اماكن مختلفه العال...
221     تمكن فريق الباحثين مختبر الفيزياء التطبيقيه ال...
702     ظل الزياده الكبيره تكاليف الامدادات والشحن وال...
999     جاء دراسه جديده صدرت قبيل المنتدي العالمي الاو...
2053    يستعد مارسيل مونستر مءسس صندوق جريتي حاليا لتق...
2872    يتطرق معرض ومءتمر الصحه العربي اراب هيلث يعد ا...
3017    ولاحظ العلماء ان النجوم تتجاوز كتلتها كتله الش...
3063    اعلنت شركه iCare Solutions اطلاق اول منصه لتتب...
4490    واكدت الدراسه المنشوره مءخرا مجله Obesity ان ف...
4641    ويكشف التقرير المعنون ماساه مهمله العبء العالم...
5560    الهمت المعادله الشهيره المستخدمه البحث حياه فض...
5690    يقول باحثون امريكيون ان فيروس كورونا ربما يتلا...
5970    وصل خبير علم الكونيات تومي تينكانين جامعه جونز...
6086    واضافت المنظمه ان السلطات البريطانيه قالت انه ...
6358    وقال موقع ميديكال اكسبرس ان باحثين بجامعه جونز...
6503    وبحسب الباحثين معهد القياسات الصحيه والتقييم ا...
7550    واوضحت

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
def cosine(query):
    vectorizer = CountVectorizer()

# Fit and transform the text data
    X = vectorizer.fit_transform(data['Content'])


    # Tokenize and vectorize the query
    query_vector = vectorizer.transform([query])

    query_vector_dense = query_vector.toarray()

    # Calculate cosine similarity between query and documents
    cosine_similarities = cosine_similarity(query_vector_dense, X)

    # Add cosine similarities to DataFrame
    data['cosine_similarity'] = cosine_similarities[0]

    # Sort DataFrame by cosine similarity
    df_sorted = data.sort_values(by='cosine_similarity', ascending=False)

    # Filter out documents with cosine similarity greater than 0
    similar_documents = df_sorted[df_sorted['cosine_similarity'] > 0.0]

    similar_document_ids = similar_documents.iloc[:, 0]

    return similar_document_ids

In [32]:
from itertools import permutations
import pandas as pd
import numpy as np
import bisect
from tkinter import *
from tkinter import ttk
from tkinter.filedialog import askopenfilename, askopenfilenames
from tkinter import messagebox
from PIL import ImageTk , Image
import os
import ast
root=Tk()
root.geometry('750x600')
root.resizable(False,False)
root.title('IR')
root.config(background='#ffffff')
img=Image.open("D://src_ir.png")
img_resize=img.resize((10,10))
icon=ImageTk.PhotoImage(img_resize)
root.iconphoto(False,icon)
docs=StringVar()
ranked_docs=StringVar()
query=StringVar()
def ranking():
    docs.set("")
    query_str = query.get()
    cosine_similarities = cosine(query_str)
    docs.set(cosine_similarities)

def search2():
    
   
    query_str = query.get()
    res1=data.iloc[search(query_str,inverted_index,data['Content'])]
    docs.set(res1.iloc[:, 0])
    
    
    
    
image=Image.open('D://src2_ir.png')
image=image.resize((750,250))
photo2=ImageTk.PhotoImage(image)
Label(root,image=photo2).place(x=0,y=0)

search_entry=Entry(root,width=55,borderwidth=2,font=("calibri",18),justify='right',textvariable=query)
search_entry.place(x=60,y=220)
image=Image.open('D://src5_ir.png')
image=image.resize((40,30))
photo3=ImageTk.PhotoImage(image)
search_btn=Button(root,width=40,height=30,image=photo3,command=search2)
search_btn.place(x=15,y=218)
frame1=Frame(root,bg="#ffffff",width=700,height=400)
frame1.place(x=10,y=270)

Label(frame1, bg="#ffffff", fg="#000000", textvariable=docs, font=("calibri", 8)).place(x=400, y=10)

Button(frame1,width=17,height=2,text="Ranking",bg='#cce7ec',command=ranking, font=("calibri", 12)).place(x=320,y=240)


root.mainloop()
